# Subsystem1 transporter


In [1]:
from biocrnpyler import *
import numpy as np
import pylab as plt

VirE2_f = Species ("VirE2_f") #, material_type = "protein") #free
repressor = Species("tetR") #, material_type = "protein")
dna_g= Species ("dna_g")
rna_g= Species ("rna_g")
ATP= Species ("ATP")
ADP= Species ("ADP")
atc_i = Species("atc_i") # material_type = "ligand")
VirE2_b = Species ("VirE2_b") #bound to membran
p = Species("P")
d_r= ComplexSpecies([dna_g,repressor])
d_r_atc= ComplexSpecies([dna_g,repressor,atc_i])



#Create a ChemicalComplex to model ligand-repressor bindning
inactive_repressor = ChemicalComplex([repressor, atc_i])

species=[dna_g, repressor, rna_g, VirE2_f, ATP,ADP, atc_i, inactive_repressor.get_species(),VirE2_b, d_r, d_r_atc, p]

rex1=Reaction([atc_i, repressor], [inactive_repressor.get_species()], propensity_type = "massaction",k= 2e-4, k_rev=2e-9) #paper Minverese secinverse k= 2e+5, k_rev=2.1
rex2=Reaction([dna_g, repressor], [d_r], propensity_type = "massaction",k= 4e-5, k_rev= 1e-12) #25
rex3=Reaction([d_r,atc_i ], [d_r_atc], propensity_type = "massaction", k= 6e-6, k_rev= 3e-9 ) #k= 1e-5, k_rev= 3e-9
rex4=Reaction([d_r_atc], [dna_g, inactive_repressor.get_species()], propensity_type = "massaction", k= 5e-3, k_rev=1e-10 ) #1000,10
rex5=Reaction([dna_g], [dna_g, rna_g], propensity_type = "massaction", k=.0031875 ) 
rex6=Reaction([rna_g, ATP], [rna_g, ADP, VirE2_f,p], propensity_type = "proportionalhillpositive", propensity_params = {"k":0.001, "n":1, "K":1, "s1":ATP, "d":rna_g}) 
rex7=Reaction([VirE2_f], [VirE2_b], propensity_type = "proportionalhillnegative", propensity_params = {"k":0.0005, "n":1, "K":1, "s1":VirE2_b, "d":VirE2_f})
rex8=Reaction([rna_g], [], propensity_type = "massaction", k= .001) 
rex9=Reaction([VirE2_f], [], propensity_type = "massaction", k= 1e-5)


r=[rex1, rex2, rex3, rex4, rex5, rex6, rex7, rex8, rex9]

CRN1 = ChemicalReactionNetwork(species = species, reactions = r)

sub1 = CRN1.write_sbml_file('transporter.xml')
print('Model written to a SBML file successfully')
#print(CRN1.pretty_print())


Model written to a SBML file successfully


 # atc external reservoir

In [3]:
##### Creating atc reservoir model####

from subsbml import createBasicSubsystem, SimpleModel
import libsbml

atc_ss, atc_model = createBasicSubsystem('external_reservoir')

per_second = atc_model.createNewUnitDefinition(uId = 'per_second',uKind = libsbml.UNIT_KIND_SECOND,
                                                uExponent = -1)
substance = atc_model.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, uExponent = 1)

atc_model.createNewSpecies('atc_e',sComp = 'external_reservoir', ListOfAmounts = 1e4, 
                             sConstant = False, sSubstance = 'substance')

# Write to SBML file 
exsub = atc_ss.writeSBML('atc_external_reservoir.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


## atc simple membrane model

In [4]:
##### Creating a basic membrane model for IPTG transport####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction

atc_mb_ss = createNewSubsystem()
model = atc_mb_ss.createNewModel('atc_membrane','second','mole','substance')

atc_mb = SimpleModel(model)

per_second = atc_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = atc_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
atc_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
atc_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

atc_mb.createNewSpecies( 'atc_e','external',0,False,'substance')
atc_mb.createNewSpecies( 'atc_i','internal',0,False,'substance')

atc_mb.createNewParameter( 'kb',2e-3,False,'per_second')
atc_mb.createNewParameter( 'kd',1e-5,False,'per_second')

r1_sbml = model.createReaction()
r1 = SimpleReaction(r1_sbml)
r1.setId('r1')
r1.setReversible(True)
r1.createNewReactant('atc_e',False,1)
r1.createNewProduct('atc_i', False, 1)
r1.createRate('kb * atc_e - kd * atc_i')

# Write to SBML file 
_ = atc_mb_ss.writeSBML('membrane_atc.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


# transporter protein advanced membrane model

In [5]:
##### Creating a advanced membrane model for DNA transport####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction

dnaexp_mb_ss = createNewSubsystem()
model = dnaexp_mb_ss.createNewModel('dnaexp_membrane','second','mole','substance')

dnaexp_mb = SimpleModel(model)

per_second = dnaexp_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = dnaexp_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
dnaexp_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
dnaexp_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

dnaexp_mb.createNewSpecies( 'ssDNA_e','external',0,False,'substance')
dnaexp_mb.createNewSpecies( 'ssDNA','internal',0,False,'substance')
dnaexp_mb.createNewSpecies( 'VirE2_b','internal',1,False,'substance')
dnaexp_mb.createNewSpecies( 'VirE2_b_ssDNA','internal',0,False,'substance')

dnaexp_mb.createNewParameter( 'kb1',0.002,False,'per_second')
dnaexp_mb.createNewParameter( 'kd1',0.00001,False,'per_second')
dnaexp_mb.createNewParameter( 'kd2',0.0002,False,'per_second')


model = dnaexp_mb_ss.getSBMLDocument().getModel()

R1 = SimpleReaction(model.createReaction())
R1.setId('R1')
R1.setReversible(True)
R1.createNewReactant('VirE2_b',False,1)
R1.createNewReactant('ssDNA', False, 1)
R1.createNewProduct('VirE2_b_ssDNA', False, 1)
R1.createRate('kb1 * VirE2_b * ssDNA - kd1 * VirE2_b_ssDNA')


R2 = SimpleReaction(model.createReaction())
R2.setId('R2')
R2.setReversible(False)
R2.createNewReactant('VirE2_b_ssDNA',False,1)
R2.createNewProduct('ssDNA_e',False,1)
R2.createNewProduct('VirE2_b',False,1)
R2.createRate('kd2 * VirE2_b_ssDNA')


# Write to SBML file 
_ = dnaexp_mb_ss.writeSBML('dnaexp_membrane1.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


# virE2 expression + ATP rheostat model


In [40]:
# Import statements
from subsbml import createNewSubsystem, createSubsystem

import numpy as np
import pylab as plt
# Import SBML models by creating Subsystem class objects
ss1 = createSubsystem('transporter.xml', subsystemName = 'virE2')
#ss2 = createSubsystem('ATP_rheostat.xml', subsystemName = 'ATP_rheostat')
ss2 = createSubsystem('ATP_SYNTHASE.xml', subsystemName = 'ATP_synthase')


ss2.renameSName('metabolite_atp', 'ATP')
ss2.renameSName('metabolite_adp', 'ADP')
ss2.renameSName('metabolite_pi', 'P')


combsub = createNewSubsystem()
combsub.combineSubsystems([ss1,ss2])

# Set signal concentration (input) - manually and get ID for protein X
_ = combsub.writeSBML('combintsub.xml')


In [41]:
ok = ss1 + ss2

In [42]:
ok.setSpeciesAmount('atc_i', 1e4)
#ok.setSpeciesAmount('ssDNA', 50)
ok.setSpeciesAmount('dna_g', 1)
ok.setSpeciesAmount('tetR', 10)
ok.setSpeciesAmount('ATP', 30)
ok.setSpeciesAmount('dna_proton_pump', 10)
ok.setSpeciesAmount('dna_atpsyn', 10)
ok.setSpeciesAmount('proton_outside_H', 10)
ok.setSpeciesAmount('protein_RNAP_machinery', 10)
ok.setSpeciesAmount('protein_Ribo_machinery', 10)
ok.setSpeciesAmount('protein_RNAse_machinery', 10)

True

In [43]:
import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

Loading BokehJS ...

In [44]:
timepoints = np.linspace(0,50000,100)
results_1, _ = ok.simulateWithBioscrape(timepoints)

ATP_id1 = ok.getSpeciesByName('ATP').getId()

odeint failed with mxstep=500...

In [45]:
p1 = bokeh.plotting.figure(width = 450, height = 350)
p1.line(timepoints, results_1[ATP_id1])
bokeh.io.show(p1)

# Models below are of different diffusion reactions (not req now)